In [45]:
import rasterio

In [46]:
import netCDF4 as nc

In [47]:
import numpy as np

In [63]:
nc_file = '/Users/choiyoonjin/Dev/geo-modelweb/Data/nc/his_west_20220703_00091.nc'

In [64]:
f = nc.Dataset(nc_file)

In [65]:
nav_lat_rho = f.variables['lat_rho'][:-1,:-1] 
nav_lon_rho = f.variables['lon_rho'][:-1,:-1]

In [66]:
lat_min = np.max(nav_lat_rho)

In [67]:
lat_min

37.98688862640065

In [68]:
lon_min = np.min(nav_lon_rho)

In [69]:
U = f.variables['u'][0,-1,:-1,:]

In [70]:
U = U[::-1,:]

In [71]:
V = f.variables['v'][0,-1,:,:-1]
V = V[::-1,:]

In [72]:
nx_rho, ny_rho = nav_lat_rho.shape[1], nav_lon_rho.shape[0]

In [73]:
lon_step = (nav_lon_rho[0,-1] - nav_lon_rho[0,0]) / nx_rho

In [74]:
lat_step = (nav_lat_rho[-1,0] - nav_lat_rho[0,0]) / ny_rho

In [86]:
wind_direction = (np.arctan2(-V,U)*180/np.pi+360)%360

In [87]:
mask = np.isnan(wind_direction)

In [82]:
with rasterio.open(
    'wind_direction6.tif',
    'w',
    driver='GTiff',
    height=U.shape[0],
    width=U.shape[1],
    count=1,
    dtype=wind_direction.dtype,
    crs='EPSG:4326',
    transform=rasterio.transform.from_origin(lon_min,lat_min,lon_step,lat_step),
) as dst:
    dst.write(wind_direction,1)
    dst.nodata = -9999

## wind speed

In [41]:
wind_speed = np.sqrt(U**2+V**2)

/Users/choiyoonjin/opt/anaconda3/envs/geomweb/lib/python3.8/site-packages/numpy/ma/core.py:6957: RuntimeWarning: overflow encountered in power
  result = np.where(m, fa, umath.power(fa, fb)).view(basetype)


In [42]:
wind_speed

masked_array(
  data=[[--, --, --, ..., --, --, --],
        [--, --, --, ..., --, --, --],
        [--, --, --, ..., --, --, --],
        ...,
        [0.12608855962753296, 0.30511316657066345, 0.3115362823009491,
         ..., --, --, --],
        [0.13179951906204224, 0.3109481632709503, 0.3644736111164093,
         ..., --, --, --],
        [0.1222233921289444, 0.1126437783241272, 0.1058214008808136, ...,
         --, --, --]],
  mask=[[ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        ...,
        [False, False, False, ...,  True,  True,  True],
        [False, False, False, ...,  True,  True,  True],
        [False, False, False, ...,  True,  True,  True]],
  fill_value=1e+37,
  dtype=float32)

In [43]:
with rasterio.open(
    'wind_speed.tif',
    'w',
    driver='GTiff',
    height=U.shape[0],
    width=U.shape[1],
    count=1,
    dtype=wind_speed.dtype,
    crs='EPSG:4326',
    transform=rasterio.transform.from_origin(lon_min,lat_min,lon_step,lat_step),
) as dst:
    dst.write(wind_speed,1)